# Press "SHIFT + ENTER" to run a cell, all cells should be run in order, from top to bottom

### First we import some libraries

In [6]:
import numpy as np
from PIL import Image

### Then we set some parameters:
1. Change 4096 to the size you want your unsig to be (width and height will be the same)

In [7]:
#pixel dimension
dim = 4096

## input your unsigs:
1. index (number)
2. number of properties
3. the values of these properties

In [8]:
#replace the content inside {} with your unsig's properties
unsig = {'index': 0,
         'num_props': 0,
         'properties': {
             'multipliers'   : [],
             'colors'        : [],
             'distributions' : [],
             'rotations'     : []}}

## Run the cell below!

In [9]:
def norm(x , mean , std):
    p = (np.pi*std) * np.exp(-0.5*((x-mean)/std)**2)
    return p

def scale_make2d(s):
    scaled = np.interp(s, (s.min(), s.max()), (0, u_range))
    two_d = np.tile(scaled, (dim, 1))
    return two_d

def gen_nft(nft):
    idx = unsig['index']
    props = unsig['properties']
    
    n = np.zeros((dim, dim, 3)).astype(np.uint32)

    for i in range(unsig['num_props']):
        mult = props['multipliers'][i]
        col = props['colors'][i]
        dist = props['distributions'][i]
        rot = props['rotations'][i]
        c = channels[col]
        buffer =  mult * np.rot90(dists[dist], k=(rot / 90))
        n[ :, :, c ] = n[ :, :, c ] + buffer

    n = np.interp(n, (0, u_range), (0, 255)).astype(np.uint8)

    return (idx, n)

if __name__ == '__main__':
    #setup
    x = list(range(dim))
    u_range = 4294967293
    mean = np.mean(x)
    std = dim/6

    #probability and cumulative distribution
    p_1d = np.array(norm(x, mean, std)).astype(np.uint32)
    c_1d = np.cumsum(p_1d)

    #2d arrays
    p_2d = scale_make2d(p_1d)
    c_2d = scale_make2d(c_1d)

    #dicts for retrieving values
    dists = {'Normal': p_2d, 'CDF': c_2d}
    channels = {'Red': 0, 'Green': 1, 'Blue': 2}
    
    #make your nft
    i, nft = gen_nft(unsig)
    
    img = Image.fromarray(nft)
    img.save(f'unsig_{i:05d}.png')